<a href="https://colab.research.google.com/github/Cru1zzz3/python-parallel-programming-cookbook/blob/main/Python_Parallel_Programming_(Lab_3)_Udartsev_Stanislav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to spawn a process**

In [4]:
import multiprocessing

def foo(i):
  print ('called function in process: %s' %i)
  return

if __name__ == '__main__':
  Process_jobs = []
  for i in range(5):
    p = multiprocessing.Process(target=foo, args=(i,))
    Process_jobs.append(p)
    p.start()
    p.join()


called function in process: 0
called function in process: 1
called function in process: 2
called function in process: 3
called function in process: 4


**How to name a process**

Процесс обращения к имени процесса схож с тем, что был во второй главе во время обращения к имени потока.

In [6]:
import multiprocessing
import time

def foo():
  name = multiprocessing.current_process().name
  print("Starting %s \n" %name)
  time.sleep(3)
  print("Exiting %s \n %name")

if __name__ == '__main__':
  process_with_name = multiprocessing.Process(name='foo_process', target=foo)
  process_with_name.daemon = True
  process_with_default_name = multiprocessing.Process(target=foo)
  process_with_name.start()
  process_with_default_name.start()

Starting foo_process 

Starting Process-19 



**How to run a process in the background**

In [12]:
import multiprocessing
import time

def foo():
 name = multiprocessing.current_process().name
 print ("Starting %s \n" %name)
 time.sleep(3)
 print ("Exiting %s \n" %name)

if __name__ == '__main__':
  background_process = multiprocessing.Process(name='background_process', target=foo)
  background_process.daemon = True

  NO_background_process = multiprocessing.Process(name='NO_background_process', target=foo)
  NO_background_process.daemon = False

  background_process.start()
  NO_background_process.start()
  
  background_process.join()
  NO_background_process.join()



Starting background_process 

Starting NO_background_process 

Exiting background_process 

Exiting NO_background_process 



**How to kill a process**

Опрос статуса процесса можно с помощью метода .is_alive(). Убить процесс с помощью метода .terminate()    

In [14]:
import multiprocessing
import time

def foo():
 print ('Starting function')
 time.sleep(0.1)
 print ('Finished function')

if __name__ ==  '__main__':
  p = multiprocessing.Process(target=foo)
  print ('Process before execution:', p, p.is_alive())
  p.start()
  print ('Process running:', p, p.is_alive())
  p.terminate()
  print ('Process terminated:', p, p.is_alive())
  p.join()
  print ('Process joined:', p, p.is_alive())
  print ('Process exit code:', p.exitcode)


Process before execution: <Process(Process-31, initial)> False
Process running: <Process(Process-31, started)> True
Process terminated: <Process(Process-31, started)> True
Process joined: <Process(Process-31, stopped[SIGTERM])> False
Process exit code: -15


**How to use a process in a subclass**

In [2]:
import multiprocessing

class MyProcess(multiprocessing.Process):
  def run(self):
    print ('called run method in process: %s' %self.name)
    return

if __name__ == '__main__':
 jobs = []
 for i in range(5):
  p = MyProcess ()
  jobs.append(p)
  p.start()
  p.join()

called run method in process: MyProcess-1
called run method in process: MyProcess-2
called run method in process: MyProcess-3
called run method in process: MyProcess-4
called run method in process: MyProcess-5


**How to exchange objects between
processes**

1. Using queue to exchange objects

In [3]:
import multiprocessing
import random
import time

class producer(multiprocessing.Process):
  def __init__(self, queue):
    multiprocessing.Process.__init__(self)
    self.queue = queue
    
  def run(self) :
    for i in range(10):
      item = random.randint(0, 256)
      self.queue.put(item)
      print ("Process Producer : item %d appended to queue %s" % (item,self.name))
      time.sleep(1)
      print ("The size of queue is %s" % self.queue.qsize())

class consumer(multiprocessing.Process):
  def __init__(self, queue):
    multiprocessing.Process.__init__(self)
    self.queue = queue

  def run(self):
    while True:
      if (self.queue.empty()):
        print("the queue is empty")
        break
      else :
        time.sleep(2)
        item = self.queue.get()
        print ('Process Consumer : item %d popped from by %s \n' % (item, self.name))
        time.sleep(1)

if __name__ == '__main__':
  queue = multiprocessing.Queue()
  process_producer = producer(queue)
  process_consumer = consumer(queue)
  process_producer.start()
  process_consumer.start()
  process_producer.join()
  process_consumer.join()



Process Producer : item 139 appended to queue producer-6
The size of queue is 1
Process Producer : item 5 appended to queue producer-6
The size of queue is 1
Process Producer : item 165 appended to queue producer-6
Process Consumer : item 139 popped from by consumer-7 

The size of queue is 2
Process Producer : item 46 appended to queue producer-6
The size of queue is 3
Process Producer : item 200 appended to queue producer-6
Process Consumer : item 5 popped from by consumer-7 
The size of queue is 3

Process Producer : item 200 appended to queue producer-6
The size of queue is 4
Process Producer : item 129 appended to queue producer-6
The size of queue is 5
Process Producer : item 191 appended to queue producer-6
Process Consumer : item 165 popped from by consumer-7 

The size of queue is 5
Process Producer : item 243 appended to queue producer-6
The size of queue is 6
Process Producer : item 85 appended to queue producer-6
The size of queue is 7
Process Consumer : item 46 popped from

2. Using pipes to exchange objects 

In [4]:
import multiprocessing

def create_items(pipe):
  output_pipe, _ = pipe
  for item in range(10):
    output_pipe.send(item)
  output_pipe.close()

def multiply_items(pipe_1, pipe_2):
  close, input_pipe = pipe_1
  close.close()
  output_pipe, _ = pipe_2
  try:
    while True:
      item = input_pipe.recv()
      output_pipe.send(item * item)
  except EOFError:
    output_pipe.close()

if __name__== '__main__':
  #First process pipe with numbers from 0 to 9
  pipe_1 = multiprocessing.Pipe(True)
  process_pipe_1 = multiprocessing.Process(target=create_items, args=(pipe_1,))
  process_pipe_1.start()
  #second pipe,
  pipe_2 = multiprocessing.Pipe(True)
  process_pipe_2 = multiprocessing.Process(target=multiply_items, args=(pipe_1, pipe_2,))
  process_pipe_2.start()
  pipe_1[0].close()
  pipe_2[0].close()
  try:
    while True:
      print(pipe_2[1].recv())
  except EOFError:
    print("End")

0
1
4
9
16
25
36
49
64
81
End


**How to synchronize processes**